## Bibliotecas

In [1]:
import pandas as pd                                     # Tratamiento de datos
import matplotlib.pyplot as plt                         # Graficos

from sklearn.model_selection import StratifiedKFold     # Clases desequelibradas
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
import statistics                                       # Estadistica


## Leer dataset

In [2]:
dataset = pd.read_excel("datosProfesores.xlsx")
dataset.head()

,Unnamed: 0,Profesores,Puntuacion,Recomiendad,Dificultad,Etiqueta
0,NaN,OCOTITLA ROJAS NANCY,4.8,0.35,3.6,regular
1,NaN,CHAVARRIA BAEZ LORENA,6.0,0.45,5.3,malo
2,NaN,RODRIGUEZ SARABIA TANIA,5.2,0.42,1.8,bueno
3,NaN,ZAGAL FLORES ROBERTO ESWART,8.0,0.90,3.3,bueno
4,NaN,CATALAN SALGADO EDGAR ARMANDO,5.5,0.48,4.2,malo


## Preprocesamiento de datos

##### Rasgos y clases

In [3]:
x = dataset.iloc[:, 2:5].values
y = dataset.iloc[:, 5].values


##### Desvalance de datos

In [4]:
# --> Contar cuantos valores hay por clase
valores_por_clase = dataset['Etiqueta'].value_counts()

# --> Buscar el valor mas grande y el mas pequeño
print(f"[+]Maximo: {max(valores_por_clase)} \n[+]Minimo: {min(valores_por_clase)}")


[+]Maximo: 36 
[+]Minimo: 22


In [5]:
ir = 36/22
print("[+]Desbalanceado" if ir > 1.5 else "[+]Balanceado")

[+]Desbalanceado


In [6]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto", k_neighbors=5)
x, y = smote.fit_resample(x, y)


In [7]:
clases_balanceadas = pd.Series(y).value_counts()
print(clases_balanceadas)

regular    36
malo       36
bueno      36
Name: count, dtype: int64


## Modelo a evaluar

In [5]:
from sklearn.svm import SVC


## Métodos de validación

##### Exactitud

In [6]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVC(), x, y, cv=10, scoring='accuracy')
print(f"[+]Exactitud: {100 * scores.mean()} %")
print(f"[+]Desviación estandar: {100 * scores.std()} %")


[+]Exactitud: 55.55555555555556 %
[+]Desviación estandar: 12.638125740085918 %


##### Validación cruzada

In [7]:
# --> Lista de metricas
lista_exactitud = []
lista_precison = []
lista_f1score = []
lista_sensibilidad = []

# --> Objeto para validacion
skf = StratifiedKFold(n_splits=10)

# ------------| Método de validación k-fold  cross-validation |------------
# --> Iteraciones
for entramiento, prueba in skf.split(x, y):
    # --> Separar datos en entrenamiento y prueba
    x_entrenamiento = [x[z] for z in entramiento]
    x_prueba = [x[z] for z in prueba]

    y_entrenamiento = [y[z] for z in entramiento]
    y_prueba = [y[z] for z in prueba]

    # --> Entrenar modelo y predecir
    y_predicha=SVC().fit(x_entrenamiento, y_entrenamiento).predict(x_prueba)

    # --> Metricas
    lista_exactitud.append(accuracy_score(y_prueba, y_predicha))
    lista_precison.append(precision_score(y_prueba, y_predicha, average="macro"))
    lista_f1score.append(f1_score(y_prueba, y_predicha, average="macro"))
    lista_sensibilidad.append(recall_score(y_prueba, y_predicha, average="macro"))

print(f"""
[+]Exactitud:\t {100*(statistics.mean(lista_exactitud))} %
[+]Presicion:\t {100*(statistics.mean(lista_precison))} %
[+]F1-score:\t {100*(statistics.mean(lista_f1score))} %
[+]Sensibilidad: {100*(statistics.mean(lista_sensibilidad))} %
""")



[+]Exactitud:	 55.55555555555556 %
[+]Presicion:	 47.51587301587302 %
[+]F1-score:	 45.01948051948052 %
[+]Sensibilidad: 49.722222222222214 %



c:\Users\uriel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\uriel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\uriel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\U

## Parámetros óptimos

In [8]:
from sklearn.model_selection import GridSearchCV

# --> Parametros a evaluar
parametros = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.5, 0.1, 0.01, 0.001, 0.0001]}
    ]

# --> Objeto de busqueda
busqueda = GridSearchCV(estimator=SVC(), param_grid=parametros, scoring='accuracy', cv=skf)

# --> Entrenamiento
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.2)

busqueda.fit(x_train, y_train)

# --> Resultados
print(f"[+]Mejor exactitud: {100*busqueda.best_score_} %")
print(f"[+]Mejores parametros: {busqueda.best_params_}")


[+]Mejor exactitud: 59.28571428571428 %
[+]Mejores parametros: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
